In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import pprint
import json

In [2]:
path1 = ("Resources/filtered_covid.csv")
path2 = ("Resources/real_estate19.csv")
path3 = ("Resources/real_estate20.csv")

covid= pd.read_csv(path1)
rs_19= pd.read_csv(path2)
rs_20= pd.read_csv(path3)

In [3]:
covid = covid[(covid['YEAR']==2020) & (covid['MONTH']>5)]
covid

,GENDER,AGEGRP,DATE,COVID_COUNT,COUNTY_NAME,YEAR,MONTH,DAY,MONTH_NAME
5,F,0-19,2020-06-11 00:00:00,1,Boone,2020,6,11,June
6,F,0-19,2020-06-19 00:00:00,1,Boone,2020,6,19,June
7,F,0-19,2020-06-20 00:00:00,1,Boone,2020,6,20,June
8,F,0-19,2020-06-22 00:00:00,1,Boone,2020,6,22,June
9,F,0-19,2020-06-25 00:00:00,1,Boone,2020,6,25,June
...,...,...,...,...,...,...,...,...,...
19438,Unknown,Unknown,2020-08-25 00:00:00,1,Marion,2020,8,25,August
19439,Unknown,Unknown,2020-08-26 00:00:00,1,Marion,2020,8,26,August
19440,Unknown,Unknown,2020-10-24 00:00:00,1,Marion,2020,10,24,October
19441,Unknown,Unknown,2020-11-13 00:00:00,1,Marion,2020,11,13,November


In [4]:
covid_county_count= covid.groupby("COUNTY_NAME")["COVID_COUNT"].sum()
covid_county_count

COUNTY_NAME
Boone         4228
Hamilton     23086
Hancock       4886
Hendricks    10502
Johnson      11150
Marion       61713
Name: COVID_COUNT, dtype: int64

In [5]:
covid_month_county= covid.groupby("MONTH_NAME")["COVID_COUNT"].sum()
covid_month_county

MONTH_NAME
August        6877
December     48027
July          5314
June          2452
November     36572
October      10884
September     5439
Name: COVID_COUNT, dtype: int64

In [6]:
covid_month_stats= covid.groupby(["COUNTY_NAME", "MONTH"])["COVID_COUNT"].sum()
covid_month_stats

COUNTY_NAME  MONTH
Boone        6          108
             7          206
             8          219
             9          163
             10         417
             11        1338
             12        1777
Hamilton     6          331
             7          956
             8         1505
             9         1063
             10        2082
             11        7542
             12        9607
Hancock      6           81
             7          187
             8          206
             9          140
             10         416
             11        1582
             12        2274
Hendricks    6          192
             7          367
             8          613
             9          454
             10         888
             11        3275
             12        4713
Johnson      6          116
             7          396
             8          406
             9          370
             10        1183
             11        3682
             12        4997
M

In [7]:
covid_county_df= pd.DataFrame(covid_county_count)
covid_county_df

,COVID_COUNT
COUNTY_NAME,
Boone,4228
Hamilton,23086
Hancock,4886
Hendricks,10502
Johnson,11150
Marion,61713


In [8]:
covid_month_df= pd.DataFrame(covid_month_stats)
covid_month_df

COVID_COUNT
COUNTY_NAME MONTH             
Boone       6              108
            7              206
            8              219
            9              163
            10             417
            11            1338
            12            1777
Hamilton    6              331
            7              956
            8             1505
            9             1063
            10            2082
            11            7542
            12            9607
Hancock     6               81
            7              187
            8              206
            9              140
            10             416
            11            1582
            12            2274
Hendricks   6              192
            7              367
            8              613
            9              454
            10             888
            11            3275
            12            4713
Johnson     6              116
            7              396
            8              406
            9              370
            10            1183
            11            3682
            12            4997
Marion      6             1624
            7             3202
            8             3928
            9             3249
            10            5898
            11           19153
            12           24659

In [29]:
#Getting mean values, and inserting into DataFrames
rs_19_mean= rs_19.groupby("MONTH")["ListPrice"].mean()
rs_20_mean= rs_20.groupby("MONTH")["ListPrice"].mean()

rs_19_price_df= pd.DataFrame(rs_19_mean)
rs_19_price_df = rs_19_price_df.rename(columns={"ListPrice": "2019 Price"})
                                    

rs_20_price_df= pd.DataFrame(rs_20_mean)
rs_20_price_df = rs_20_price_df.rename(columns={"ListPrice": "2020 Price"})

#Combining Real Estate into single DataFrame
rs_combined_price = pd.merge(rs_19_price_df,rs_20_price_df, on="MONTH")
rs_combined_price

#Formatting for currency
rs_combined_price = rs_combined_price.round(decimals = 2)
rs_combined_price

,2019 Price,2020 Price
MONTH,,
6,234036.04,284110.68
7,233241.49,280252.04
8,221376.34,266243.75
9,213293.07,252501.72
10,189435.05,239411.54
11,209368.01,235463.85
12,267263.87,278015.52


In [41]:
#Looking at counties
rs19_county_mean= rs_19.groupby(["County", "MONTH"])["ListPrice"].mean()
rs20_county_mean= rs_20.groupby(["County", "MONTH"])["ListPrice"].mean()

rs19_county_df= pd.DataFrame(rs19_county_mean)
rs19_county_df = rs19_county_df.rename(columns={"ListPrice": "2019 Price"})
rs19_county_df

rs20_county_df= pd.DataFrame(rs20_county_mean)
rs20_county_df = rs20_county_df.rename(columns={"ListPrice": "2020 Price"})
rs20_county_df

rs_combined_county = pd.merge(rs19_county_df,rs20_county_df, on=["MONTH", "County"])
rs_combined_county

#Formatting for currency
rs_combined_county = rs_combined_county.round(decimals = 2)
rs_combined_county

,,2019 Price,2020 Price
MONTH,County,,
6,Boone,388336.03,415895.80
7,Boone,302653.16,363381.13
8,Boone,304809.97,400380.03
9,Boone,288279.03,383739.58
10,Boone,381970.16,347987.95
11,Boone,245146.00,412390.69
12,Boone,292910.83,420559.29
6,Hamilton,318732.75,379091.42
7,Hamilton,326852.76,379281.31


In [53]:
rs19_monthly_count= rs_19.groupby(["County", "MONTH"])["MLSNumber"].count()
rs19_monthly_count = pd.DataFrame(rs19_monthly_count)
rs19_monthly_count = rs19_monthly_count.rename(columns={"MLSNumber": "2019 Listings"})
rs19_monthly_count

rs20_monthly_count= rs_20.groupby(["County", "MONTH"])["MLSNumber"].count()
rs20_monthly_count = pd.DataFrame(rs20_monthly_count)
rs20_monthly_count = rs20_monthly_count.rename(columns={"MLSNumber": "2020 Listings"})
rs20_monthly_count

#Combining into singular DataFrame
rs_combined_listing = pd.merge(rs19_monthly_count,rs20_monthly_count, on=["MONTH", "County"])
rs_combined_listing

,,2019 Listings,2020 Listings
MONTH,County,,
6,Boone,117,155
7,Boone,101,144
8,Boone,96,108
9,Boone,80,118
10,Boone,62,84
11,Boone,17,61
12,Boone,6,38
6,Hamilton,669,800
7,Hamilton,685,860
